In [ ]:
import os
import cv2
from transform_image import process
import numpy as np
import random
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.regularizers import l2
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ELU
from keras.optimizers import Adam
from pyimagesearch.callbacks import EpochCheckpoint
from pyimagesearch.callbacks import TrainingMonitor
from sklearn.model_selection import train_test_split

import matplotlib
matplotlib.use("Agg")

In [2]:
#Skin tone
lower = np.array([0, 48, 80], dtype = "uint8")
upper = np.array([20, 255, 255], dtype = "uint8")
    
def mask(x):
    converted = cv2.cvtColor(x, cv2.COLOR_BGR2HSV)
    skinMask = cv2.inRange(converted, lower, upper)
    skinMask = cv2.GaussianBlur(skinMask, (7, 7), 0)
    return skinMask

In [3]:
#Path to get and save images
dPath = "./Data/"

#Folder names
folders = ["forks", "left", "stop", "peace", "right"]

#Gesture dictionary
gesture = {'forks':0,'left':1,'stop':2,'peace':3, 'right':4}

#Train Test data storage
X_train = []
y_train =[]

In [4]:
# im = cv2.imread(dPath + folders[0] + "/" + "forks(1).jpg")
# im = cv2.resize(im, dsize=(50,50))
# im = mask(im)
# cv2.imshow('im',im)
# cv2.waitKey()

In [5]:
broke = 0
#Load in our training data with a testtrain split
for folder in folders:
    listing = os.listdir(dPath + folder)
    for file in listing:
        #Import and process image into mask
        im = cv2.imread(dPath + folder + "/" + file)
            
        if im is not None:
            im = cv2.resize(im, dsize=(50,50))
            im = mask(im)

            im = np.asarray(im)/255
            im = np.expand_dims(im,axis=2)
            X_train.append(im)
            y_val = np.zeros(len(folders))
            y_val[gesture[folder]] = 1
            y_train.append(y_val)
        
        else: 
            broke = broke + 1

In [6]:
#Convert lists to numpy arrays
X = np.asarray(X_train)
y = np.asarray(y_train)

In [7]:
inputShape = (50,50,1)
chanDim = 2
reg = l2(0.001)

# Start model
model = Sequential()

# Block #1: first CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(32, (3, 3), padding="same",kernel_initializer="he_normal", input_shape=inputShape))
model.add(ELU())
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(32, (3, 3), kernel_initializer="he_normal",padding="same"))
model.add(ELU())
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Block #2: second CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(64, (3, 3), kernel_initializer="he_normal",padding="same"))
model.add(ELU())
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), kernel_initializer="he_normal",padding="same"))
model.add(ELU())
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Block #3: third CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(128, (3, 3), kernel_initializer="he_normal",padding="same"))
model.add(ELU())
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), kernel_initializer="he_normal",padding="same"))
model.add(ELU())
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Block #4: first set of FC => RELU layers
model.add(Flatten())
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(ELU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block #6: second set of FC => RELU layers
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(ELU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block #7: softmax classifier
model.add(Dense(5, kernel_initializer="he_normal"))
model.add(Activation("softmax"))

#Create the model
#model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])
opt = Adam(lr=1e-3)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
epoch = 0
# construct the set of callbacks
figPath = os.path.sep.join(["outputs","vggnet_emotion.png"])
jsonPath = os.path.sep.join(["outputs","vggnet_emotion.json"])
callbacks = [EpochCheckpoint("checkpoints", every=5,startAt=epoch),TrainingMonitor(figPath, jsonPath=jsonPath,startAt=epoch)]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

#Train the model
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=40, shuffle=True, callbacks=callbacks, batch_size=128)

Instructions for updating:
Use tf.cast instead.
Train on 19952 samples, validate on 8552 samples
Epoch 1/40
19952/19952 [==============================] - 379s 19ms/step - loss: 0.9592 - acc: 0.6500 - val_loss: 0.3340 - val_acc: 0.8845
Epoch 2/40
19952/19952 [==============================] - 369s 18ms/step - loss: 0.3891 - acc: 0.8702 - val_loss: 0.1728 - val_acc: 0.9426
Epoch 3/40
19952/19952 [==============================] - 374s 19ms/step - loss: 0.2573 - acc: 0.9176 - val_loss: 0.1275 - val_acc: 0.9559
Epoch 4/40
19952/19952 [==============================] - 354s 18ms/step - loss: 0.1990 - acc: 0.9384 - val_loss: 0.1039 - val_acc: 0.9681
Epoch 5/40
19952/19952 [==============================] - 364s 18ms/step - loss: 0.1576 - acc: 0.9507 - val_loss: 0.1028 - val_acc: 0.9689
Epoch 6/40
19952/19952 [==============================] - 387s 19ms/step - loss: 0.1370 - acc: 0.9574 - val_loss: 0.0957 - val_acc: 0.9723
Epoch 7/40
19952/19952 [==============================] - 424s 21ms/s

In [10]:
#Save the model
model.save("processed28k(pyimage)5hrs.h5py")